In [1]:
import os
import pandas as pd
from yahoo_fin.stock_info import get_data
from concurrent.futures import ThreadPoolExecutor

In [2]:
# load files
transactions = pd.read_csv('../../data/outputs/insider_marked_trades.csv')

In [3]:
# read from text file
file = open('../../data/inputs/bad_tickers.txt', 'r')
# get the list of bad tickers from file, separated by a comma
bad_tickers = file.read().split(',')
file.close()

# drop the bad_tickers from the transactions
transactions = transactions[~transactions['ticker'].isin(bad_tickers)]

In [4]:
# if a person sells a stock and there is no buy transaction before that, then we should remove that sell transaction

def remove_transactions_without_buy(transactions):
    # sort the transactions by name, ticker and transaction_date
    transactions = transactions.sort_values(['name', 'ticker', 'transaction_date'])
    # group the transactions by name and ticker
    grouped = transactions.groupby(['name', 'ticker'])
    # create a new dataframe to store the transactions that we want to keep
    new_transactions = pd.DataFrame(columns=transactions.columns)
    # purchase flag
    # iterate through the groups
    for name, group in grouped:
        # print(f'Processing {name}...{group.shape[0]} transactions...')
        # set the purchase flag to False
        purchased = False
        # iterate through the transactions in the group
        for index, transaction in group.iterrows():
            # if the transaction is a buy transaction, set the purchase flag to True
            if transaction['type'] == 'purchase':
                purchased = True
                # add the transaction to the new_transactions
                new_transactions = pd.concat([new_transactions, pd.DataFrame(transaction).T])
            # if the transaction is a sell (sell_partial or sell_full) transaction AND the purchase flag is True, add the transaction to the new_transactions
            elif purchased and transaction['type'] in ['sale_partial', 'sale_full']:
                new_transactions = pd.concat([new_transactions, pd.DataFrame(transaction).T])

    return new_transactions

transactions_processed = remove_transactions_without_buy(transactions)

In [5]:
# replace $1,001 - with $1,001 - $15,000 in the amount column
transactions_processed['amount'] = transactions_processed['amount'].str.strip()
transactions_processed['amount'] = transactions_processed['amount'].replace('$1,001 -', '$1,001 - $15,000')

# create a new column amount_average by taking the average of amount column in each row
# amount: $1,001 - $15,000 --> amount_average = (1001 + 15000) / 2 = 8000.5
for index, row in transactions_processed.iterrows():
    # print(f'Processing {index}...')
    # get the amount range
    amount_range = row['amount']
    # split the amount range by ' - '
    amount_range = amount_range.split(' - ')
    # get the first and last elements of the amount range
    first = int(amount_range[0].replace('$', '').replace(',', ''))
    last = int(amount_range[1].replace('$', '').replace(',', ''))
    # calculate the average
    average = (first + last) / 2
    # set the average to the amount_average column
    transactions_processed.at[index, 'amount_average'] = average


In [10]:
# DIFFERENTIATOR: remove all trades that are not insider
transactions_processed = transactions_processed[transactions_processed['is_insider'] == True]

In [13]:
earliest_date = transactions_processed['transaction_date'].min()
earliest_date

'2015-01-13'

In [14]:
# df is transactions_processed[name = some_name]
def get_stock_data(ticker_list, transaction_df):
    data = pd.DataFrame()
    end_date = '2024-04-29'
    for ticker in ticker_list:
        try: 
            # start date is the date of the first transaction
            start_date = transaction_df[transaction_df['ticker'] == ticker]['transaction_date'].min()
            df = get_data(ticker=ticker, start_date=start_date, end_date=end_date, 
                          index_as_date=False, interval='1d')
            df = df[['date', 'ticker', 'adjclose']]
            # calculate daily return
            df['daily_ret'] = df['adjclose'].pct_change()
            data = pd.concat([data, df], axis=0)
        except:
            print(f'Error getting data for {ticker}...')
            continue
        
    data['date'] = pd.to_datetime(data['date'])
    return data

In [15]:
from datetime import date, timedelta

def get_portfolio_return(toy, data):
    # get the first row in the toy dataframe
    start_date = toy.loc[0, 'transaction_date'].split('-')
    start_date = date(int(start_date[0]), int(start_date[1]), int(start_date[2]))
    end_date = date(2024, 4, 29)

    # columns = stocks + cash + total
    portfolio = pd.DataFrame(columns=list(toy.ticker.unique()) + ['cash', 'total', 'cash_flow', 'return', 'cum_return'],
                                index=pd.date_range(start_date, end_date))
    # fill na with float(0)
    portfolio = portfolio.fillna(float(0))

    # iterate over each row in portfolio
    for index, row in portfolio.iterrows():
        # get the date
        current_date = index.strftime('%Y-%m-%d')
        # cash flow for calculating the daily return
        portfolio.at[index, 'cash_flow'] = 0
        # add daily return to the current position for each stock
        for ticker in list(toy.ticker.unique()):
            # get the previous value
            previous = 0
            daily_ret = 0
            # if it's not the start date, get the value of each stock on the previous day
            if current_date != start_date.strftime('%Y-%m-%d'):
                previous = portfolio.at[index - timedelta(days=1), ticker]
            # get the daily return if it exists in the data
            if data[(data['date'] == current_date) & (data['ticker'] == ticker)].shape[0] > 0:
                daily_ret = data[(data['date'] == current_date) & (data['ticker'] == ticker)]['daily_ret'].values[0]
            # calculate the new value for the stock's position
            portfolio.at[current_date, ticker] = previous * (1 + daily_ret)
        # cash position is the same as the previous day
        if current_date != start_date.strftime('%Y-%m-%d'):
            portfolio.at[index, 'cash'] = portfolio.at[index - timedelta(days=1), 'cash']
        
        # get the transactions that happened on that date
        transactions = toy[toy['transaction_date'] == current_date]
        # iterate over each transaction
        for i, transaction in transactions.iterrows():
            amount = transaction['amount_average']
            ticker = transaction['ticker']
            # if the transaction is a purchase transaction
            if transaction['type'] == 'purchase':
                # update the portfolio
                portfolio.at[index, ticker] += amount
                # update cash
                portfolio.at[index, 'cash'] = max(0, portfolio.at[index, 'cash'] - amount)
                # update cash flow
                if amount > portfolio.at[index, 'cash']:
                    portfolio.at[index, 'cash_flow'] += amount
                    
            # if the transaction is a sell transaction, subtract the amount from the current position
            # transfer that amount to the cash position
            elif transaction['type'] in ['sale_partial', 'sale_full']:
                if portfolio.at[index, ticker] >= amount:
                    portfolio.at[index, ticker] -= amount
                    portfolio.at[index, 'cash'] += amount
                    # total remains the same
                else:
                    actual_amount = portfolio.at[index, ticker]
                    portfolio.at[index, ticker] = 0
                    portfolio.at[index, 'cash'] += actual_amount
                    
    # calculate the total value of the portfolio for each date
    portfolio['total'] = portfolio[list(toy.ticker.unique())].sum(axis=1) + portfolio['cash']
    # calculate the daily return: ( V_t - V_(t-1) - CF_t) / (V_(t-1) + CF_t
    portfolio['return'] = (portfolio['total'] - portfolio['total'].shift(1) - portfolio['cash_flow']) / (portfolio['total'].shift(1) + portfolio['cash_flow'])
    # calculate the cumulative return
    portfolio['cum_return'] = (1 + portfolio['return']).cumprod() - 1
    return portfolio

In [17]:
def process_portfolio(name, transactions):
    # Filter and sort transactions for the person
    toy = transactions[(transactions['name'] == name)]
    toy = toy.reset_index(drop=True).sort_values('transaction_date').reset_index(drop=True)
    # Get unique stock tickers
    stocks = toy['ticker'].unique()
    # Fetch stock data
    data = get_stock_data(stocks, toy)
    data = data.fillna(0)
    # Compute portfolio returns
    portfolio = get_portfolio_return(toy, data)
    # Generate filename and save to CSV
    file_name = '_'.join(name.split(' ')) + '.csv'
    portfolio.to_csv(f'../../data/working_files/insider_trade_returns/{file_name}', index=True)

# get the names of all the people in the transactions_processed
names = transactions_processed['name'].unique()

DATA_PATH = '../../data'

if not os.path.exists(DATA_PATH+'/working_files/insider_trade_returns'):
    os.makedirs(DATA_PATH+'/working_files/insider_trade_returns')

# iterate through the names
with ThreadPoolExecutor() as executor:
        # Submit tasks to the thread pool
        results = executor.map(lambda name: process_portfolio(name, transactions_processed), names)

In [18]:
# loop through all the files in the working_files/insider_trade_returns/ directory
# get its index (date), return and cum_return columns
# combine it into a single dataframe: name, date, return, cum_return

# names = ['Nancy Pelosi', 'Tom Rice']
# create a new dataframe to store the results
results = pd.DataFrame(columns=['name', 'date', 'return', 'cum_return'])
# iterate through the files in the working_files directory
# file name is names split by ' ' and joined by '_'
for name in names:
    file_name = '_'.join(name.split(' ')) + '.csv'
    # read the file
    file = pd.read_csv(f'../../data/working_files/insider_trade_returns/{file_name}', index_col=0)
    # turn index into a column
    file['date'] = file.index
    # get the columns: date, return, cum_return
    file = file[['date', 'return', 'cum_return']]
    # add the name column
    file['name'] = name
    # append the file to the results
    results = pd.concat([results, file])
    
results = results.fillna(0)
results = results.reset_index(drop=True)
# save the results to a csv file
results.to_csv('../../data/outputs/insider_returns.csv', index=False)
results.to_csv('../../data/outputs/congress_returns.csv.gz', index=False, compression='gzip')

/var/folders/px/62m8qql13zqcl_2xc5d532nc0000gn/T/ipykernel_75733/3891226228.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, file])


In [19]:
results.head()

,name,date,return,cum_return
0,"Angus S. King, Jr.",2021-06-24,0.000000,0.000000
1,"Angus S. King, Jr.",2021-06-25,0.002421,0.002421
2,"Angus S. King, Jr.",2021-06-26,0.000000,0.002421
3,"Angus S. King, Jr.",2021-06-27,0.000000,0.002421
4,"Angus S. King, Jr.",2021-06-28,0.015768,0.018227


In [20]:
results.groupby('date')['cum_return'].mean()

date
2015-01-13    0.000000
2015-01-14   -0.003811
2015-01-15   -0.030847
2015-01-16   -0.038378
2015-01-17   -0.038378
                ...   
2024-04-25    0.382090
2024-04-26    0.401246
2024-04-27    0.401246
2024-04-28    0.401246
2024-04-29    0.401246
Name: cum_return, Length: 3395, dtype: float64